In [16]:
!pip install pinecone==6.0.1 pinecone-notebooks

In [17]:
import os
if not os.environ.get("PINECONE_API_KEY"):
   from pinecone_notebooks.colab import Authenticate
   Authenticate()

In [18]:
from pinecone import Pinecone
api_key = os.environ["PINECONE_API_KEY"]
environment = "us-west1-gcp"
pc = Pinecone(api_key=api_key, environment=environment)

In [30]:
query = "Tell me about Apple's products"

documents = [
    "Apple Inc. recently released the new iPhone with improved battery life.",
    "I love starting my day with a fresh green apple and a cup of tea.",
    "The Apple Watch Series 9 includes new health monitoring features.",
    "Granny Smith apples are known for their tart flavor and crisp texture.",
    "Apple's MacBook Pro is popular among software developers for its performance."
]

In [28]:
from pinecone import RerankModel

reranked = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=query,
    documents=[{"id": str(i), "text": doc} for i, doc in enumerate(documents)],
    top_n=3
)

In [29]:
def show_reranked(query, data):
    print(f"Query: {query}\n")
    for i, item in enumerate(data):
        score = item['score']
        text = item['document']['text']
        print(f"Rank {i+1}: score = {score:.4f} | text = {text}")


# 5. Afficher les résultats
show_reranked(query, reranked.data)

Query: Tell me about Apple's fruit

Rank 1: score = 0.0061 | text = Granny Smith apples are known for their tart flavor and crisp texture.
Rank 2: score = 0.0015 | text = I love starting my day with a fresh green apple and a cup of tea.
Rank 3: score = 0.0003 | text = Apple's MacBook Pro is popular among software developers for its performance.


In [31]:
!pip install pandas torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [83]:
import os, time, pandas as pd, torch
from pinecone import Pinecone, ServerlessSpec

# Environnement Pinecone
cloud = "aws"
region = "us-east-1"

# Configuration serverless (exemple : 1 CPU, 2 Go RAM)
spec = ServerlessSpec(cloud=cloud, region=region)

# Nom de ton index Pinecone
index_name = "pinecone-reranker"

# Connexion au client Pinecone
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

# Création de l'index si besoin
if index_name not in pc.list_indexes().names():
    pc.create_index(name=index_name, dimension=768, spec=spec)

print(f"✅ Connected to Pinecone index: {index_name}")

✅ Connected to Pinecone index: pinecone-reranker


In [84]:
# Exemple : modèle d'embedding qui retourne des vecteurs de dimension 384
dimension = 384

if pc.has_index(index_name):
    pc.delete_index(index_name)

pc.create_index(
    name=index_name,
    dimension=dimension,
    spec=spec
)

print(f"✅ Index '{index_name}' créé avec dimension {dimension}")

✅ Index 'pinecone-reranker' créé avec dimension 384


In [85]:
import os
import requests
import tempfile
import pandas as pd

with tempfile.TemporaryDirectory() as tmpdir:
    file_path = os.path.join(tmpdir, "sample_notes_data.jsonl")
    url = "https://raw.githubusercontent.com/pinecone-io/examples/refs/heads/master/docs/data/sample_notes_data.jsonl"
    resp = requests.get(url)
    resp.raise_for_status()
    open(file_path, "wb").write(resp.content)

    # Lecture du fichier JSONL ligne par ligne
    df = pd.read_json(file_path, orient='records', lines=True)

print(df.head())

     id                                             values  \
0  P011  [-0.2027486265, 0.2769146562, -0.1509393603, 0...   
1  P001  [0.1842793673, 0.4459365904, -0.0770567134, 0....   
2  P002  [-0.2040648609, -0.1739618927, -0.2897160649, ...   
3  P003  [0.1889383644, 0.2924542725, -0.2335938066, -0...   
4  P004  [-0.12171068040000001, 0.1674752235, -0.231888...   

                                            metadata  
0  {'advice': 'rest, hydrate', 'symptoms': 'heada...  
1  {'tests': 'EKG, stress test', 'symptoms': 'che...  
2  {'HbA1c': '7.2', 'condition': 'diabetes', 'med...  
3  {'symptoms': 'cough, wheezing', 'diagnosis': '...  
4  {'referral': 'dermatology', 'condition': 'susp...  


In [87]:
index = pc.Index(index_name)
index.upsert_from_dataframe(df)

sending upsert requests:   0%|          | 0/100 [00:00<?, ?it/s]

{'upserted_count': 100}

In [94]:
def is_ready(idx):
   stats = idx.describe_index_stats()
   return stats.total_vector_count > 0

while not is_ready(index):
   time.sleep(5)
print(index.describe_index_stats())

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 100}},
 'total_vector_count': 100,
 'vector_type': 'dense'}


In [99]:
from sentence_transformers import SentenceTransformer

def get_embedding(text):
   model = SentenceTransformer("all-MiniLM-L6-v2")
   return model.encode(text).tolist()

In [106]:
# Préparer la liste de vecteurs à insérer
vectors = []

for _, row in df.iterrows():
    # Construct a text string from available metadata fields
    metadata_text = ""
    if isinstance(row["metadata"], dict):
        # You might need to adjust these keys based on the actual metadata structure
        if "symptoms" in row["metadata"]:
            metadata_text += f"Symptoms: {row['metadata']['symptoms']}. "
        if "diagnosis" in row["metadata"]:
            metadata_text += f"Diagnosis: {row['metadata']['diagnosis']}. "
        if "advice" in row["metadata"]:
            metadata_text += f"Advice: {row['metadata']['advice']}. "
        if "condition" in row["metadata"]:
             metadata_text += f"Condition: {row['metadata']['condition']}. "
        # Add other relevant keys from your metadata here

    vectors.append({
        "id": row["id"],
        "values": row["values"],
        "metadata": {
            "text": metadata_text.strip() # Use the constructed text
        }
    })

# Insérer dans Pinecone
index.upsert(vectors=vectors)
print(f"✅ Upserté {len(vectors)} vecteurs dans Pinecone.")

✅ Upserté 100 vecteurs dans Pinecone.


In [107]:
# Requête clinique en langage naturel
question = "What if my patient has leg pain?"

# Calcul de l'embedding à partir de la question
emb = get_embedding(question)

# Connexion à l'index Pinecone
index = pc.Index(index_name)

# Recherche des 5 notes les plus similaires
results = index.query(
    vector=emb,
    top_k=5,
    include_metadata=True
)

# Trier les résultats par score décroissant
matches = sorted(results.matches, key=lambda m: m.score, reverse=True)

# Afficher les résultats
for i,m in enumerate(matches,1):
    print(f"{i}. id={m.id}, score={m.score:.4f}, metadata={m.metadata}")



1. id=P0100, score=0.5329, metadata={'text': 'Symptoms: muscle pain. Advice: over-the-counter pain relief, stretching.'}
2. id=P095, score=0.5078, metadata={'text': 'Symptoms: back pain.'}
3. id=P047, score=0.5078, metadata={'text': 'Symptoms: back pain.'}
4. id=P007, score=0.4539, metadata={'text': 'Symptoms: pain, swelling.'}
5. id=P044, score=0.4475, metadata={'text': 'Condition: dehydration.'}


In [109]:
# Suppose que "matches" contient des objets avec un .id et un .metadata
rerank_docs = [
    {
        "id": m.id,
        "reranking_field": "; ".join([f"{k}: {v}" for k, v in m.metadata.items()])
    }
    for m in matches
]

# Question plus précise, par exemple dans un domaine médical :
rerank_query = "What treatments are recommended for patients with severe asthma symptoms?"


In [113]:
reranked = pc.inference.rerank(
   model="bge-reranker-v2-m3",
   query=rerank_query,
   documents=rerank_docs,
   rank_fields=["reranking_field"],
   top_n=2
)

In [114]:
def show_reranked(q, matches):
    print(f"Refined Query: {q}\n")
    for i, m in enumerate(matches):
        rank = i + 1
        doc_id = m.document["id"]
        score = m.score
        content = m.document["reranking_field"]
        print(f"Rank {rank}: id = {doc_id} | score = {score:.4f} | content = {content}")


In [115]:
show_reranked(rerank_query, reranked.data)

Refined Query: What treatments are recommended for patients with severe asthma symptoms?

Rank 1: id = P0100 | score = 0.0049 | content = text: Symptoms: muscle pain. Advice: over-the-counter pain relief, stretching.
Rank 2: id = P007 | score = 0.0030 | content = text: Symptoms: pain, swelling.
